# Transverse Momentum Plots

This Jupyter notebook contains generators which polar plots the transverse momenta of the particles involved in an event versus the angle. You see 3 events with all an incoming neutrino of a different flavour at once. You can choose which event you want and at which energy you want to see it.

In [1]:
import matplotlib.pyplot as plt
import numpy as np

import km3pipe as kp
import km3modules as km
import km3io
from km3net_testdata import data_path
from particle import Particle
import ipywidgets as iw

%matplotlib widget

In [2]:
file_electron = "/dcache/antares/detector_data/mc/atm-neutrinos/V7.00/generator/KM3NeT_00000049/mcv7.0.gsg_elec-CC_100-500GeV.133.root"
file_anti_electron = "/dcache/antares/detector_data/mc/atm-neutrinos/V7.00/generator/KM3NeT_00000049/mcv7.0.gsg_anti-elec-CC_100-500GeV.107.root" 
file_muon = "/dcache/antares/detector_data/mc/atm-neutrinos/V7.00/generator/KM3NeT_00000049/mcv7.0.gsg_muon-NC_100-500GeV.144.root"
file_anti_muon = "/dcache/antares/detector_data/mc/atm-neutrinos/V7.00/generator/KM3NeT_00000049/mcv7.0.gsg_anti-muon-CC_100-500GeV.127.root"
file_tau = "/dcache/antares/detector_data/mc/atm-neutrinos/V7.00/generator/KM3NeT_00000049/mcv7.0.gsg_tau-CC_100-300GeV.142.root"
file_anti_tau = "/dcache/antares/detector_data/mc/atm-neutrinos/V7.00/generator/KM3NeT_00000049/mcv7.0.gsg_anti-tau-CC_100-300GeV.132.root"

file_electron = "/dcache/antares/detector_data/mc/atm-neutrinos/V7.00/generator/KM3NeT_00000049/mcv7.0.gsg_elec-CC_1-100GeV.140.root"
file_muon = "/dcache/antares/detector_data/mc/atm-neutrinos/V7.00/generator/KM3NeT_00000049/mcv7.0.gsg_muon-CC_1-100GeV.140.root"
file_tau = "/dcache/antares/detector_data/mc/atm-neutrinos/V7.00/generator/KM3NeT_00000049/mcv7.0.gsg_tau-CC_3-100GeV.1005.root"

f_e = km3io.OfflineReader(data_path(file_electron))
f_m = km3io.OfflineReader(data_path(file_muon))
f_t = km3io.OfflineReader(data_path(file_tau))


# Functions

In [3]:
def particles_momentum(f_event):
    vectors_final = rotation_particles(f_event)
    
    momentum = get_momentum(f_event.mc_tracks)
    
    vectors_final[0] = vectors_final[0]*momentum
    vectors_final[1] = vectors_final[1]*momentum
    vectors_final[2] = vectors_final[2]*momentum
    
    return vectors_final

In [12]:
def polar(x, y, z):
    #r = np.sqrt(x**2+y**2+z**2)
    theta = np.arccos(z/r)
    theta = np.nan_to_num(theta)
    
    y_mask1 = (y>=0)
    
    phi1 = np.arccos(x/np.sqrt(x**2+y**2))
    phi_array1 = phi1*y_mask1
       
    y_mask2 = (y<0)
    phi2 = 2*np.pi - np.arccos(x/np.sqrt(x**2+y**2))
    phi_array2 = phi2*y_mask2
    
    phi = phi_array1+phi_array2
    phi = np.nan_to_num(phi)
    
    return r, theta, phi

In [5]:
def rotation_particles(f_event):
    pos = tuple(np.zeros(f_event.n_mc_trks))
    dir_x = tuple(f_event.mc_tracks.dir_x)
    dir_y = tuple(f_event.mc_tracks.dir_y)
    dir_z = tuple(f_event.mc_tracks.dir_z)
    
    vector = np.array([dir_x[0], dir_y[0], dir_z[0]])
    
    vector_without_y = np.array([vector[0], 0, vector[2]])    
    angle_from_z = kp.math.angle(vector_without_y, np.array([1,0,0]))
    y_axis = np.array([0,1,0])
    
    if vector[2] < 0:
        angle_from_z = (2*np.pi-angle_from_z)
    rotation_matrix_from_z = kp.math.rotation_matrix(y_axis,angle_from_z)
    
    vector_new = rotation_matrix_from_z@vector
    
    vector_without_z = np.array([vector_new[0], vector_new[1], 0])
    angle_from_y = kp.math.angle(vector_without_z, np.array([1,0,0]))
    z_axis = np.array([0,0,1])
    
    if vector_new[1] > 0:
        angle_from_y = (2*np.pi-angle_from_y)
    rotation_matrix_from_y = kp.math.rotation_matrix(z_axis,angle_from_y)
    
    vector_final = rotation_matrix_from_y@vector_new
    
    vectors = np.array([dir_x, dir_y, dir_z])
    vectors_new = rotation_matrix_from_z@vectors
    vectors_final = rotation_matrix_from_y@vectors_new
    
    return vectors_final

In [6]:
def get_momentum(mc_tracks):
    masses = get_masses(mc_tracks) #MeV
    Es = mc_tracks.E #GeV
    Es_MeV = Es * 10**3 #MeV
    momentum = np.sqrt(Es_MeV**2 - masses**2)
    momentum = np.nan_to_num(momentum) * 10**-3
    return momentum

In [7]:
def get_masses(mc_tracks):
    pdg_ids = mc_tracks.pdgid
    masses = np.zeros(len(pdg_ids))
    i=0
    for pdg in pdg_ids:
        p = Particle.from_pdgid(pdg)
        mass = p.mass
        if mass == None:
            mass = 0
        masses[i]=mass
        i=i+1
    return masses

# Polar Transverse Momentum Plot Generator

In [11]:
E_e_sort = np.sort(np.around((f_e.mc_tracks.E)[:,0]))
index_e = np.argsort(np.around((f_e.mc_tracks.E)[:,0]))
E_e_sort2 = E_e_sort[E_e_sort!=1]
E_e_sort3 = E_e_sort2[E_e_sort2!=2]
index_e = index_e[E_e_sort!=1]
index_e = index_e[E_e_sort2!=2]

E_m_sort = np.sort(np.around((f_m.mc_tracks.E)[:,0]))
index_m = np.argsort(np.around((f_m.mc_tracks.E)[:,0]))
E_m_sort2 = E_m_sort[E_m_sort!=1]
E_m_sort3 = E_m_sort2[E_m_sort2!=2]
index_m = index_m[E_m_sort!=1]
index_m = index_m[E_m_sort2!=2]


E_t_sort = np.sort(np.around((f_t.mc_tracks.E)[:,0]))
index_t = np.argsort(np.around((f_t.mc_tracks.E)[:,0]))
E_t_sort2 = E_t_sort[E_t_sort!=1]
E_t_sort3 = E_t_sort2[E_t_sort2!=2]
index_t = index_t[E_t_sort!=1]
index_t = index_t[E_t_sort2!=2]

opts = dict(continuous_update=False)
@iw.interact
def transverse_momentum(event=iw.BoundedIntText(value=0,min=0,max=100,description="Event"),
                        energy=iw.IntSlider(value=3,min=3,max=20)):
    index_e_energy = index_e[E_e_sort3==energy]
    momentum1 = particles_momentum(f_e[index_e_energy[event]])
    r_mom1, theta_mom1, phi_mom1 = polar(momentum1[1],momentum1[2],momentum1[0])
    momentum_t1 = np.sqrt(momentum1[1]**2 + momentum1[2]**2)
    E1 = round(f_e[index_e_energy[event]].mc_tracks.E[0])

    index_m_energy = index_m[E_m_sort3==energy]
    momentum2 = particles_momentum(f_m[index_m_energy[event]])
    r_mom2, theta_mom2, phi_mom2 = polar(momentum2[1],momentum2[2],momentum2[0])
    momentum_t2 = np.sqrt(momentum2[1]**2 + momentum2[2]**2)
    E2 = round(f_m[index_m_energy[event]].mc_tracks.E[0])

    index_t_energy = index_t[E_t_sort3==energy]
    momentum3 = particles_momentum(f_t[index_t_energy[event]])
    r_mom3, theta_mom3, phi_mom3 = polar(momentum3[1],momentum3[2],momentum3[0])
    momentum_t3 = np.sqrt(momentum3[1]**2 + momentum3[2]**2)
    E3 = round(f_t[index_t_energy[event]].mc_tracks.E[0])

    fig = plt.figure()
    ax = fig.add_subplot(projection='polar')
    c = ax.scatter(np.delete(phi_mom1,1), np.delete(momentum_t1,1), color='blue', label=f"Electron, E = {E1} GeV")
    c = ax.scatter(np.delete(phi_mom2,1), np.delete(momentum_t2,1), color='green', label=f"Muon, E = {E2} GeV")
    c = ax.scatter(np.delete(phi_mom3,1), np.delete(momentum_t3,1), color='red', label=f"Tau, E = {E3} GeV")
    plt.legend(bbox_to_anchor=(1.35, 1), loc="right")
    plt.xlabel("phi")
    ax.set_rlim(0,1)
    plt.show()    

interactive(children=(BoundedIntText(value=0, description='Event'), IntSlider(value=3, description='energy', m…